# LDA Recommendation System

In [7]:
import pandas as pd
import numpy as np
import nltk
import re
import ast
import gensim
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter
from nltk.tokenize import word_tokenize
from gensim import corpora

In [8]:
job_data = pd.read_csv('job_data_sahy.csv')
resume_data = pd.read_csv('ResumswithoutLanguage&Location.csv')

In [9]:
# for col in job_data.columns:
#     print(col, ":     " , job_data[col].iloc[0])

# print(resume_data['skills'].iloc[0][0])

In [10]:
resume_data['skills'].iloc[0]

"['Hdfs', 'Core Java', 'Hive', 'Talend', 'YARN', 'Microsoft Azure', 'Java', 'Hadoop', 'Data Structures', 'Machine Learning', 'Cloud', 'Amazon Redshift', 'RDBMS', 'Python', 'Informatica', 'Data Science', 'C++', 'Spark', 'Mapreduce', 'DBMS', 'Big Data']"

In [11]:
# Clean Skills
def get_skills(skill_string):
    skill_set = []
    s = ""
    f = False
    for char in skill_string:
        if char == r"'" and f == False:
            f = True
            s = ""
        elif char == r"'" and f == True:
            f = False
            skill_set.append(s)
            s = ""
        else:
            s += char
    return skill_set

resume_data['clean skills'] = resume_data['skills'].apply(get_skills)
resume_data['clean skills']

0       [Hdfs, Core Java, Hive, Talend, YARN, Microsof...
1       [R, SAS, Data Visualization, Business Analytic...
2       [HTML, Hive, PySpark, Application Support, Uni...
3       [commercial auto underwriting, mortgage loan, ...
4       [administrative, audit procedures, general led...
                              ...                        
9539    [Machine Learning, Artificial Intelligence, Py...
9540    [R, Data Visualization, Regression Analysis, D...
9541    [Machine Learning, Data Cleansing, Deep Learni...
9542    [Machine Learning, Sentiment Analysis, GPT, Te...
9543    [marketing, networking, Team player, time mana...
Name: clean skills, Length: 9544, dtype: object

In [12]:
all_skills = {}
for skill_set in resume_data['clean skills']:
    for skill in skill_set:
        if skill in all_skills.keys():
            all_skills[skill] += 1
        else:
            all_skills[skill] = 1

print("COUNT : ", len(all_skills))
all_skills

COUNT :  4541


{'Hdfs': 28,
 'Core Java': 28,
 'Hive': 168,
 'Talend': 28,
 'YARN': 28,
 'Microsoft Azure': 112,
 'Java': 1092,
 'Hadoop': 140,
 'Data Structures': 336,
 'Machine Learning': 3444,
 'Cloud': 140,
 'Amazon Redshift': 28,
 'RDBMS': 84,
 'Python': 3640,
 'Informatica': 28,
 'Data Science': 924,
 'C++': 1064,
 'Spark': 168,
 'Mapreduce': 28,
 'DBMS': 112,
 'Big Data': 196,
 'R': 644,
 'SAS': 168,
 'Data Visualization': 588,
 'Business Analytics': 364,
 'PowerBi': 84,
 'Data Analytics': 728,
 'Data Analysis': 1680,
 'Tableau': 840,
 'Business Analysis': 448,
 'HTML': 532,
 'PySpark': 392,
 'Application Support': 56,
 'Unified Payment Interface': 28,
 'CSS': 336,
 'Software Development': 420,
 'JavaScript': 336,
 'Docker': 168,
 'Deep Learning': 1400,
 'Requirement Gathering': 196,
 'C': 756,
 'Risk Assessment': 56,
 'Risk Prediction': 28,
 'commercial auto underwriting': 28,
 'mortgage loan': 28,
 'credit checks': 28,
 'insurance': 336,
 'QuickBooks': 448,
 'Access': 588,
 'clients': 560,
 

In [13]:
# all_jobs = {}
# for job in resume_data['job_position_name']:
#     if job in all_jobs.keys():
#         all_jobs[job] += 1
#     else:
#         all_jobs[job] = 1
            
# print("COUNT : ", len(all_jobs))
# all_jobs

In [14]:
# clean_resume = resume_data.drop('clean skills', axis=1).drop_duplicates()

In [15]:
resume_data['description'] = resume_data['qualification'] + resume_data['work_history'] + resume_data['skills']
resume_data['description']

0       big data analytics working and database wareho...
1       fresher looking to join as a data analyst and ...
2       , Birla Institute of Technology (BIT), Ranchi,...
3       to obtain a position in a fastpaced business o...
4       professional accountant with an outstanding wo...
                              ...                        
9539    , Sanghvi College of Engineering, B.Tech, N/AP...
9540    expertise eda modeler i like to learn what my ...
9541    looking for roles related to application devel...
9542    , Rajiv Gandhi Memorial University, Delhi, B.T...
9543    , ARIZONA STATE UNIVERSITY, B.A, Organizationa...
Name: description, Length: 9544, dtype: object

In [16]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt_tab')

en_stopwords = stopwords.words('english')
en_stopwords.remove("not")
lm = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\everest\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\everest\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\everest\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [17]:
def preprocess_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'\d+', '', text)  # Remove numbers
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    tokens = word_tokenize(text)  # Tokenize words
    tokens = [word for word in tokens if word not in stopwords.words("english")]  # Remove stopwords
    tokens = [lm.lemmatize(word) for word in tokens]  # Lemmatizing
    return tokens

resume_data["processed_text"] = resume_data["description"].apply(preprocess_text)
# resume_data['processed_text']

job_data['processed_text'] = job_data['Long Description'].apply(preprocess_text)

In [18]:
job_data['processed_text'].iloc[0]

['social',
 'medium',
 'manager',
 'oversee',
 'organization',
 'social',
 'medium',
 'presence',
 'create',
 'schedule',
 'content',
 'engage',
 'follower',
 'analyze',
 'social',
 'medium',
 'metric',
 'drive',
 'brand',
 'awareness',
 'engagement',
 'digital',
 'marketing',
 'specialist',
 'social',
 'medium',
 'manager',
 'social',
 'medium',
 'platform',
 'eg',
 'facebook',
 'twitter',
 'instagram',
 'content',
 'creation',
 'scheduling',
 'social',
 'medium',
 'analytics',
 'insight',
 'community',
 'engagement',
 'paid',
 'social',
 'advertising',
 'manage',
 'grow',
 'social',
 'medium',
 'account',
 'create',
 'engaging',
 'content',
 'interact',
 'online',
 'community',
 'develop',
 'social',
 'medium',
 'content',
 'calendar',
 'strategy',
 'monitor',
 'social',
 'medium',
 'trend',
 'engagement',
 'metric']

In [19]:
# Create dictionary
dictionary = corpora.Dictionary(job_data["processed_text"])

# Convert text into Bag of Words representation
corpus = [dictionary.doc2bow(text) for text in job_data["processed_text"]]

# Save dictionary and corpus for later use
dictionary.save("lda_dictionary.dict")
corpora.MmCorpus.serialize("lda_corpus.mm", corpus)

In [20]:
# Train LDA model
lda_model = gensim.models.LdaModel(
    corpus=corpus, 
    id2word=dictionary, 
    num_topics=10,  # Adjust based on performance
    passes=15, 
    random_state=42
)

# Print topics
for idx, topic in lda_model.print_topics():
    print(f"Topic {idx}: {topic}")

Topic 0: 0.051*"sale" + 0.021*"environmental" + 0.018*"development" + 0.014*"team" + 0.013*"hr" + 0.012*"manager" + 0.012*"software" + 0.012*"management" + 0.011*"resource" + 0.011*"system"
Topic 1: 0.022*"health" + 0.021*"benefit" + 0.020*"therapy" + 0.019*"assessment" + 0.018*"landscape" + 0.016*"sustainability" + 0.016*"sustainable" + 0.016*"employee" + 0.015*"therapist" + 0.015*"child"
Topic 2: 0.046*"care" + 0.034*"product" + 0.030*"patient" + 0.026*"healthcare" + 0.022*"nurse" + 0.019*"medical" + 0.018*"nursing" + 0.018*"pediatric" + 0.015*"provide" + 0.012*"art"
Topic 3: 0.088*"financial" + 0.041*"investment" + 0.024*"management" + 0.020*"strategy" + 0.019*"planning" + 0.018*"client" + 0.017*"risk" + 0.016*"portfolio" + 0.015*"analysis" + 0.014*"provide"
Topic 4: 0.073*"data" + 0.031*"system" + 0.026*"design" + 0.025*"analysis" + 0.022*"engineer" + 0.020*"database" + 0.019*"research" + 0.018*"analyst" + 0.013*"performance" + 0.012*"business"
Topic 5: 0.042*"marketing" + 0.037*"n

In [21]:
def get_topic_distribution(text):
    bow_vector = dictionary.doc2bow(preprocess_text(text))  # Convert text to bag-of-words
    return lda_model.get_document_topics(bow_vector, minimum_probability=0.0)  # Get topic distribution

# Apply function to all job descriptions
job_data["topic_distribution"] = job_data["Long Description"].apply(lambda x: get_topic_distribution(x))

from sklearn.metrics.pairwise import cosine_similarity

def get_resume_vector(resume_text):
    bow_vector = dictionary.doc2bow(preprocess_text(resume_text))
    topic_vector = lda_model.get_document_topics(bow_vector, minimum_probability=0.0)
    return np.array([prob for _, prob in topic_vector]).reshape(1, -1)

# Example resume
user_resume = "Software engineer with Python, machine learning, and data analysis experience."

# Convert resume to LDA topic space
user_vector = get_resume_vector(user_resume)

# Convert all job postings to a NumPy matrix
job_topic_matrix = np.array([[prob for _, prob in topic] for topic in job_data["topic_distribution"]])

# Compute similarity
similarities = cosine_similarity(user_vector, job_topic_matrix)

In [29]:
# Get top 5 recommended jobs
job_data["similarity_score"] = similarities[0]
recommended_jobs = job_data.sort_values(by="similarity_score", ascending=False)

# Print recommendations
recommended_jobs[['Job Title', 'Job Description', 'similarity_score']].head(n=20)

,Job Title,Job Description,similarity_score
633,architect,architectural designers create architectural p...,0.999229
19464,architect,architectural designers create architectural p...,0.999229
16924,architect,architectural designers create architectural p...,0.999229
6271,architect,architectural designers create architectural p...,0.999228
11098,architect,architectural designers create architectural p...,0.999228
8142,architect,architectural designers create architectural p...,0.999228
11403,architect,architectural designers create architectural p...,0.999228
13587,architect,architectural designers create architectural p...,0.999228
3707,architect,architectural designers create architectural p...,0.999228
11514,architect,architectural designers create architectural p...,0.999228


# BM25

In [23]:
# pip install --upgrade pip
# !pip install bm25okapi
# !pip install pybm25
!pip install rank_bm25
!pip install pybm25

ERROR: Could not find a version that satisfies the requirement pybm25 (from versions: none)
ERROR: No matching distribution found for pybm25


In [32]:
from rank_bm25 import BM25Okapi
import nltk
# from nltk.tokenize import BM25

# Tokenize job descriptions
job_data["bm25_tokens"] = job_data["Long Description"].apply(lambda x: preprocess_text(x))

In [1]:
from rank_bm25 import BM25Okapi
import nltk
start = time.time()
bm25 = BM25Okapi(job_data["bm25_tokens"].tolist())

# Process user's resume
user_resume = "Software engineer with Python, machine learning, and cloud computing experience."
user_resume_tokens = preprocess_text(user_resume)

# Get BM25 similarity scores
bm25_scores = bm25.get_scores(user_resume_tokens)

# Add scores to dataframe
job_data["bm25_score"] = bm25_scores

# Get top 5 recommended jobs
recommended_jobs_bm25 = job_data.sort_values(by="bm25_score", ascending=False).head(5)
end = time.time()

print("Total Time: ", end - start)

# Print recommendations
recommended_jobs_bm25.head(n=5)

NameError: name 'time' is not defined

In [52]:
import time
start = time.time()
user_resume = "Software engineer with Python, machine learning, and cloud computing experience."
user_resume_tokens = preprocess_text(user_resume)

# Get BM25 similarity scores
bm25_scores = bm25.get_scores(user_resume_tokens)

# Add scores to dataframe
job_data["bm25_score"] = bm25_scores

# Get top 5 recommended jobs
recommended_jobs_bm25 = job_data.sort_values(by="bm25_score", ascending=False).head(5)
end = time.time()
print("Total Time: ", end - start)

# Print recommendations
recommended_jobs_bm25.head(n=5)

Total Time:  0.10068392753601074


,Job Id,Experience,Qualifications,Salary Range,location,Country,latitude,longitude,Company Size,Job Posting Date,...,Male_Preference,Female_Preference,Long Description,Preprocessed Description,Top Words,processed_text,topic_distribution,similarity_score,bm25_tokens,bm25_score
327,1140429647542292,4 to 9,ba,55000-111000,hagatna,guam,13.4443,144.7937,81941,2022-06-11,...,True,False,machine learning engineers develop machine lea...,"['machine', 'learning', 'engineer', 'develop',...","['learning', 'and', 'machine', 'data', 'model'...","[machine, learning, engineer, develop, machine...","[(0, 0.0019621574), (1, 0.0019617542), (2, 0.0...",0.974931,"[machine, learning, engineer, develop, machine...",23.515521
2657,2931542319659305,4 to 8,bca,65000-108000,dodoma,tanzania,-6.3690,34.8888,28385,2022-10-22,...,False,True,machine learning engineers develop machine lea...,"['machine', 'learning', 'engineer', 'develop',...","['learning', 'and', 'machine', 'data', 'model'...","[machine, learning, engineer, develop, machine...","[(0, 0.0019621747), (1, 0.0019617542), (2, 0.0...",0.974933,"[machine, learning, engineer, develop, machine...",23.515521
11866,854967665256911,3 to 15,bca,59000-118000,lusaka,zambia,-13.1339,27.8493,51587,2023-05-02,...,False,True,machine learning engineers develop machine lea...,"['machine', 'learning', 'engineer', 'develop',...","['learning', 'and', 'machine', 'data', 'model'...","[machine, learning, engineer, develop, machine...","[(0, 0.0019621586), (1, 0.0019617542), (2, 0.0...",0.974931,"[machine, learning, engineer, develop, machine...",23.515521
2152,1373350638609181,2 to 10,ba,62000-105000,apia,samoa,-13.7590,-172.1046,31009,2023-01-24,...,True,False,machine learning engineers develop machine lea...,"['machine', 'learning', 'engineer', 'develop',...","['learning', 'and', 'machine', 'data', 'model'...","[machine, learning, engineer, develop, machine...","[(0, 0.0019621814), (1, 0.001961754), (2, 0.00...",0.974932,"[machine, learning, engineer, develop, machine...",23.515521
9357,452900945203119,1 to 15,mca,56000-129000,bishkek,kyrgyz republic,41.2044,74.7661,52480,2021-10-18,...,False,True,machine learning engineers develop machine lea...,"['machine', 'learning', 'engineer', 'develop',...","['learning', 'and', 'machine', 'data', 'model'...","[machine, learning, engineer, develop, machine...","[(0, 0.0019621612), (1, 0.001961754), (2, 0.00...",0.974932,"[machine, learning, engineer, develop, machine...",23.515521


In [9]:
import pandas as pd
resume_sample = pd.read_json("resume_response.json")
resume_sample.loc['url']['resume']

'http://localhost:8000/uploads/resumes/resume-214523687-1739274957397.pdf'